In [5]:
from pytorch3d import io
import torch
from pytorch3d.renderer.cameras import FoVPerspectiveCameras, PerspectiveCameras
from pytorch3d.renderer.points.rasterizer import PointsRasterizer, PointsRasterizationSettings
from pytorch3d.structures import Pointclouds
from pytorch3d.utils import cameras_from_opencv_projection 
from pytorch3d.renderer import (
    PointsRenderer,
    NormWeightedCompositor
)

from scene.dataset_readers import readColmapSceneInfo
from utils.camera_utils import loadCam
from collections import namedtuple
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
import mrob
from tqdm import tqdm
from joblib import Parallel, delayed
import os
import pickle
import cv2
import pandas as pd
import copy
from glob import glob
import json

In [6]:
def compute_point_cloud_camera_fraction(R, tvec, fx, fy, cx, cy, height, width, points, build_image=False):
    camera_matrix = torch.tensor([[fx, 0, cx],
                               [0, fy, cy],
                               [0,0,1]])
    # R = [camera.R for camera in cameras]
    # tvec = [camera.T for camera in cameras]

    camera_p3d = cameras_from_opencv_projection(R = torch.tensor(np.array(R)).unsqueeze(0).float(), 
                                                tvec = torch.tensor(np.array(tvec)).unsqueeze(0).float(), 
                                                camera_matrix = camera_matrix.unsqueeze(0).float(),
                                                image_size = torch.tensor([height, 
                                                                          width]).unsqueeze(0).float())
    
    raster_settings = PointsRasterizationSettings(
                    image_size=(height, 
                                width), 
                    radius = 0.025,
                    points_per_pixel = 1 
                    )

    # Create a points rasterizer
    

    rasterizer = PointsRasterizer(cameras=camera_p3d.cuda(), raster_settings=raster_settings)
    rasterized = rasterizer(points)

    image = None
    if build_image:
        renderer = PointsRenderer(
            rasterizer=rasterizer,
            # Pass in background_color to the alpha compositor, setting the background color 
            # to the 3 item tuple, representing rgb on a scale of 0 -> 1, in this case blue
            compositor=NormWeightedCompositor()
        )

        image = renderer(points)

    fraction_set = set(torch.unique(rasterized.idx)[1:].tolist())

    return fraction_set, image

def compute_iou_2sets(set0, set1):

    intersection_indices = set0.intersection(set1)
    union_indices = set0.union(set1)
    iou = len(intersection_indices)/len(union_indices)

    return iou, list(intersection_indices), list(union_indices)

In [7]:
data_sparse = pd.read_csv('/mnt/sdb1/home/kbotashev/iros_paper/ibr_dataset/results/sparse_results_top5.csv')

In [31]:
room_name = 'drjohnson'
ply_path_base = "/mnt/sdb1/home/kbotashev/iros_paper/ibr_dataset/scenes/$roomname$/$roomname$_base/sparse/0/points3D.ply"
ply_path = ply_path_base.replace('$roomname$', room_name)
task_dir_path = os.path.join(ply_path.replace('_base/sparse/0/points3D.ply', '_task'))
base_dir_path = os.path.join(ply_path.replace('_base/sparse/0/points3D.ply', '_base'))

# scene_info_task = readColmapSceneInfo(task_dir_path, 'images', eval=False)
scene_info_base = readColmapSceneInfo(base_dir_path, 'images', eval=True)

pcd_o3d = o3d.io.read_point_cloud(ply_path)
pcd_o3d = pcd_o3d.voxel_down_sample(voxel_size=0.025)
points_tensor = torch.tensor(np.asarray(pcd_o3d.points)).unsqueeze(0).cuda().float()
colors_tensor = torch.tensor(np.asarray(pcd_o3d.colors)).unsqueeze(0).cuda().float()
points = Pointclouds(points_tensor, features=colors_tensor)

args = namedtuple('args', ['resolution', 'data_device'])
args = args(1, 'cuda:1')
pipe = namedtuple('pipe', ['convert_SHs_python', 'compute_cov3D_python', 'debug'])
pipe = pipe(False, False, False)
bg_color = [0, 0, 0]
background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")

ids = np.arange(len(scene_info_base.test_cameras))

data_sparse_room = data_sparse[data_sparse['room_name'] == room_name]

for id in tqdm(ids):
    cam_info = scene_info_base.test_cameras[id]
    camera = loadCam(args=args, id = id, cam_info=cam_info, resolution_scale=1)

    height = camera.image_height
    width = camera.image_width
    fx = camera.fx
    fy = camera.fy
    cx = camera.cx
    cy = camera.cy
    scene_results = {}
    scene_results['point_cloud_path'] = ply_path
    scene_results['scene'] = ply_path.split('/')[-5]
    scene_results['img_name'] = cam_info.image_name
    frame_results = {0:{}}

    T_gt = mrob.geometry.SE3(mrob.geometry.SO3(camera.R), camera.T)
    set_gt, _ = compute_point_cloud_camera_fraction(**{'R':T_gt.R(), 'tvec':T_gt.t(), 
                            "fx":fx, 'fy':fy, 
                            'cx':cy, 'cy':cy, 
                            'height':height, 'width':width, 
                            'points':points})
    
    row = data_sparse_room[data_sparse_room['frame_id'] == cam_info.image_name]

    top5_frames = row['top_k_refs'].item().replace('[', '').replace('\n', '').replace(']', '').replace("'", '').split(' ')[:5]

    init_ids = [topk_path.split('/')[-1].split('.jpg')[0] for topk_path in top5_frames]

    scene_info_base_train_cameras = {scene_info_base.train_cameras[i].image_name:scene_info_base.train_cameras[i] for i in range(len(scene_info_base.train_cameras))}

    for int_id, init_id in enumerate(init_ids):

        camera_init = loadCam(args=args, id = int_id, cam_info=scene_info_base_train_cameras[init_id], resolution_scale=1)

        set_init, image_init = compute_point_cloud_camera_fraction(**{'R':camera_init.R, 'tvec':camera_init.T, 
                        "fx":fx, 'fy':fy, 
                        'cx':cy, 'cy':cy, 
                        'height':height, 'width':width, 
                        'points':points,
                        'build_image':True})
        
        norm_image = cv2.normalize(image_init[0].cpu().numpy(), None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)

        norm_image = norm_image.astype(np.uint8)

        iou, intersection_indices, union_indices = compute_iou_2sets(set_gt, set_init)

        frame_results[0][init_id] = {'R_init':camera_init.R,
                                'T_init':camera_init.T,
                                'iou':iou,
                                'init_fraction_set_idx':list(set_init),
                                'intersection_idx':intersection_indices,
                                'union_idx':union_indices,
                                'init_image':norm_image,
                                'init_base_id':init_id,
                                'black_pixels_ratio':0
                                                        }
                
        # init_frames_stats = {key:len(frame_results[key]) for key in frame_results.keys()}    
    
    scene_results['frames'] = {'img_path':cam_info.image_path,
                                'R':cam_info.R,
                                'T':cam_info.T,
                                'uid':id,
                                'FovY':cam_info.FovY,
                                'FovX':cam_info.FovX,
                                'image':np.array(cam_info.image),
                                'width':cam_info.width,
                                'height':cam_info.height,
                                'qvec':cam_info.qvec,
                                'cx':cam_info.cx,
                                'cy':cam_info.cy,
                                'gt_fraction_set_idx':list(set_gt),
                                'init_frames_iou_bins':frame_results,
                                # 'init_frames_iou_bins_stats':init_frames_stats
                                }
    
    os.makedirs(os.path.join(task_dir_path, 'task-base_images_pairs'), exist_ok = True)
    with open(os.path.join(task_dir_path, 'task-base_images_pairs', cam_info.image_name + '.pickle'), 'wb') as handle:
        pickle.dump(scene_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

print('---FINISHED---')


Reading camera 263/263


100%|██████████| 33/33 [00:17<00:00,  1.85it/s]

---FINISHED---


In [32]:
tasks_root = '/mnt/sdb1/home/kbotashev/iros_paper/ibr_dataset/results/tasks'

for room in ['drjohnson', 'playroom']:
    for camera_type in ['lietorch']:
    # for camera_type in ['lietorch', 'qtvec']:
        room_task_info = {  
                            'room_name':room,
                            # 'camera_type':camera_type,
                            # # 'camera_type':'qtvec',
                            # 'solving_method':'vanilla',
                            # 'solving_method_args': None,
                            # 'loss_type':'l1',
                            # 'optimizer_type':'adam',
                            # 'init_render_resolution':2,
                            # 'iterations':2000,
                            # 'exit_psnr_parameter':1e-4,
                            # 'pose_lr_init':0.01,
                            # 'pose_lr_final':1e-5,
                            # 'pose_lr_delay_steps':0,
                            # 'pose_lr_delay_mult':0,
                            'start_frame':0,
                            'last_frame':32,
                            'experiments_base_dir':'/mnt/sdb1/home/kbotashev/iros_paper/ibr_dataset/'
                        }

        ply_path = os.path.join(room_task_info['experiments_base_dir'], 
                                'output', room_task_info['room_name'], 
                                'point_cloud/iteration_40000/point_cloud.ply')

        frames_info_pickle_paths = sorted(glob(os.path.join(room_task_info['experiments_base_dir'], 
                                        'scenes', room_task_info['room_name'], 
                                        room_task_info['room_name']+'_task', 'task-base_images_pairs/*.pickle')))

        for frames_info_pickle_path in tqdm(frames_info_pickle_paths[room_task_info['start_frame']:room_task_info['last_frame']]):
            frame_name = frames_info_pickle_path.split('/')[-1].split('.pickle')[0]

            with open(frames_info_pickle_path, 'rb') as handle:
                frame_info = pickle.load(handle)

            for init_id in frame_info['frames']['init_frames_iou_bins'][0].keys():
                task_info = copy.copy(room_task_info)
                del task_info['start_frame']
                del task_info['last_frame']

                pickle_path = os.path.join(room_task_info['experiments_base_dir'], 
                                    'scenes', room_task_info['room_name'], 
                                    room_task_info['room_name']+'_task',
                                    'task-base_images_pairs',  frame_name + '.pickle')
                
                task_info['ply_path'] = ply_path
                task_info['pickle_path'] = pickle_path
                task_info['init_id'] = init_id
                task_info['iou_bin'] = 0
                task_info['frame_name'] = frame_name

                room_task_dir = os.path.join(tasks_root, task_info['room_name'])

                iou = frame_info['frames']['init_frames_iou_bins'][0][init_id]['iou']

                frame_experiment_task_dir = os.path.join(room_task_dir, frame_name, ('%.2f'%0).replace('.', ''))

                os.makedirs(frame_experiment_task_dir, exist_ok=True)

                id_iou_frame_json_name = str(init_id).zfill(3) + '_' + ('%.2f'%iou).replace('.', '') + '_' +  frame_name + '.json'

                json_path = os.path.join(frame_experiment_task_dir, id_iou_frame_json_name)


                with open(json_path, 'w') as f:
                    json.dump(task_info, f, indent=4)

100%|██████████| 29/29 [00:00<00:00, 105.53it/s]


In [3]:
room_name = 'apartment_1'
ply_path_base = "/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/scenes/$roomname$/$roomname$_base/sparse/0/points3D.ply"
ply_path = ply_path_base.replace('$roomname$', room_name)
task_dir_path = os.path.join(ply_path.replace('_base/sparse/0/points3D.ply', '_task'))
base_dir_path = os.path.join(ply_path.replace('_base/sparse/0/points3D.ply', '_base'))

scene_info_task = readColmapSceneInfo(task_dir_path, 'images', eval=False)
scene_info_base = readColmapSceneInfo(base_dir_path, 'images', eval=False)

pcd_o3d = o3d.io.read_point_cloud(ply_path)
pcd_o3d = pcd_o3d.voxel_down_sample(voxel_size=0.025)
points_tensor = torch.tensor(np.asarray(pcd_o3d.points)).unsqueeze(0).cuda().float()
colors_tensor = torch.tensor(np.asarray(pcd_o3d.colors)).unsqueeze(0).cuda().float()
points = Pointclouds(points_tensor, features=colors_tensor)

args = namedtuple('args', ['resolution', 'data_device'])
args = args(2, 'cuda:1')
pipe = namedtuple('pipe', ['convert_SHs_python', 'compute_cov3D_python', 'debug'])
pipe = pipe(False, False, False)
bg_color = [0, 0, 0]
background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")

ids = np.arange(len(scene_info_task.train_cameras))

data_sparse_room = data_sparse[data_sparse['room_name'] == room_name]

for id in tqdm(ids):
    cam_info = scene_info_task.train_cameras[id]
    camera = loadCam(args=args, id = id, cam_info=cam_info, resolution_scale=1)

    height = camera.image_height
    width = camera.image_width
    fx = camera.fx
    fy = camera.fy
    cx = camera.cx
    cy = camera.cy
    scene_results = {}
    scene_results['point_cloud_path'] = ply_path
    scene_results['scene'] = ply_path.split('/')[-5]
    scene_results['img_name'] = cam_info.image_name
    frame_results = {0:{}}

    T_gt = mrob.geometry.SE3(mrob.geometry.SO3(camera.R), camera.T)
    set_gt, _ = compute_point_cloud_camera_fraction(**{'R':T_gt.R(), 'tvec':T_gt.t(), 
                            "fx":fx, 'fy':fy, 
                            'cx':cy, 'cy':cy, 
                            'height':height, 'width':width, 
                            'points':points})
    
    row = data_sparse_room[data_sparse_room['frame_id'] == id]

    top5_frames = row['top_k_refs'].item().replace('[', '').replace('\n', '').replace(']', '').replace("'", '').split(' ')[:5]


    init_ids = [int(topk_path.split('/')[-1].split('.png')[0]) for topk_path in top5_frames]

    for init_id in init_ids:

        camera_init = loadCam(args=args, id = init_id, cam_info=scene_info_base.train_cameras[init_id], resolution_scale=1)

        set_init, image_init = compute_point_cloud_camera_fraction(**{'R':camera_init.R, 'tvec':camera_init.T, 
                        "fx":fx, 'fy':fy, 
                        'cx':cy, 'cy':cy, 
                        'height':height, 'width':width, 
                        'points':points,
                        'build_image':True})
        
        norm_image = cv2.normalize(image_init[0].cpu().numpy(), None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)

        norm_image = norm_image.astype(np.uint8)

        iou, intersection_indices, union_indices = compute_iou_2sets(set_gt, set_init)

        frame_results[0][init_id] = {'R_init':camera_init.R,
                                'T_init':camera_init.T,
                                'iou':iou,
                                'init_fraction_set_idx':list(set_init),
                                'intersection_idx':intersection_indices,
                                'union_idx':union_indices,
                                'init_image':norm_image,
                                'init_base_id':init_id,
                                'black_pixels_ratio':0
                                                        }
                
        # init_frames_stats = {key:len(frame_results[key]) for key in frame_results.keys()}    
    
    scene_results['frames'] = {'img_path':cam_info.image_path,
                                'R':cam_info.R,
                                'T':cam_info.T,
                                'uid':id,
                                'FovY':cam_info.FovY,
                                'FovX':cam_info.FovX,
                                'image':np.array(cam_info.image),
                                'width':cam_info.width,
                                'height':cam_info.height,
                                'qvec':cam_info.qvec,
                                'cx':cam_info.cx,
                                'cy':cam_info.cy,
                                'gt_fraction_set_idx':list(set_gt),
                                'init_frames_iou_bins':frame_results,
                                # 'init_frames_iou_bins_stats':init_frames_stats
                                }
    
    os.makedirs(os.path.join(task_dir_path, 'task-base_images_pairs'), exist_ok = True)
    with open(os.path.join(task_dir_path, 'task-base_images_pairs', cam_info.image_name + '.pickle'), 'wb') as handle:
        pickle.dump(scene_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

print('---FINISHED---')


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/sdb1/home/kbotashev/iros_paper/ibr_dataset/scenes/playroom/playroom_task/sparse/0/images.txt'

In [7]:
tasks_root = '/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/results_task_rebase_vladnet/tasks'

for room in ['apartment_1', 'office_2']:
    for camera_type in ['lietorch']:
    # for camera_type in ['lietorch', 'qtvec']:
        room_task_info = {  
                            'room_name':room,
                            # 'camera_type':camera_type,
                            # # 'camera_type':'qtvec',
                            # 'solving_method':'vanilla',
                            # 'solving_method_args': None,
                            # 'loss_type':'l1',
                            # 'optimizer_type':'adam',
                            # 'init_render_resolution':2,
                            # 'iterations':2000,
                            # 'exit_psnr_parameter':1e-4,
                            # 'pose_lr_init':0.01,
                            # 'pose_lr_final':1e-5,
                            # 'pose_lr_delay_steps':0,
                            # 'pose_lr_delay_mult':0,
                            'start_frame':0,
                            'last_frame':32,
                            'experiments_base_dir':'/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/'
                        }

        ply_path = os.path.join(room_task_info['experiments_base_dir'], 
                                'output', room_task_info['room_name'], 
                                'point_cloud/iteration_40000/point_cloud.ply')

        frames_info_pickle_paths = sorted(glob(os.path.join(room_task_info['experiments_base_dir'], 
                                        'scenes', room_task_info['room_name'], 
                                        room_task_info['room_name']+'_task', 'task-rebase_images_pairs/*.pickle')))

        for frames_info_pickle_path in tqdm(frames_info_pickle_paths[room_task_info['start_frame']:room_task_info['last_frame']]):
            frame_name = frames_info_pickle_path.split('/')[-1].split('.pickle')[0]

            with open(frames_info_pickle_path, 'rb') as handle:
                frame_info = pickle.load(handle)

            for init_id in frame_info['frames']['init_frames_iou_bins'][0].keys():
                task_info = copy.copy(room_task_info)
                del task_info['start_frame']
                del task_info['last_frame']

                pickle_path = os.path.join(room_task_info['experiments_base_dir'], 
                                    'scenes', room_task_info['room_name'], 
                                    room_task_info['room_name']+'_task',
                                    'task-rebase_images_pairs',  frame_name + '.pickle')
                
                task_info['ply_path'] = ply_path
                task_info['pickle_path'] = pickle_path
                task_info['init_id'] = init_id
                task_info['iou_bin'] = 0
                task_info['frame_name'] = frame_name

                room_task_dir = os.path.join(tasks_root, task_info['room_name'])

                iou = frame_info['frames']['init_frames_iou_bins'][0][init_id]['iou']

                frame_experiment_task_dir = os.path.join(room_task_dir, frame_name, ('%.2f'%0).replace('.', ''))

                os.makedirs(frame_experiment_task_dir, exist_ok=True)

                id_iou_frame_json_name = str(init_id).zfill(3) + '_' + ('%.2f'%iou).replace('.', '') + '_' +  frame_name + '.json'

                json_path = os.path.join(frame_experiment_task_dir, id_iou_frame_json_name)


                with open(json_path, 'w') as f:
                    json.dump(task_info, f, indent=4)

100%|██████████| 32/32 [00:00<00:00, 49.75it/s]


In [33]:
step_frame = 1
room_names = ['office_2', 'apartment_1']
frame_num = np.arange(32)
camera_types = ['lietorch']
solving_methods = ['coarse']
init_render_resolutions = [2]

# step_frame = 1
# room_names = ['apartment_0', 'office_0', 'office_2', 'apartment_1', 'office_4']
# frame_num = np.arange(32)
# camera_types = ['qtvec', 'lietorch']
# solving_methods = ['vanilla', 'coarse']

# loss_types=['l1']
# optimizer_types=['adam']
# init_render_resolutions=[2]
# iterations_params=[2000]
# exit_psnr_parameters=[5e-5],
# pose_lrs_init=[0.01]
# pose_lrs_final=[1e-5]
# pose_lrs_delay_steps=[0]
# pose_lrs_delay_mult=[0]

# solving_methods_args = [{ 
#                         'max_scale':2,
#                         'num_tries':5,
#                         'blur_2d_c2f_kernel_size':201,
#                         'blur_2d_c2f_schedule':[0.05, 0.025, 0.0125, 0.00625, 0.00625, 0.0, 0.0, 0.0, 0.0, 0.0],
#                        }]

step_frame = 1
room_names = ['drjohnson', 'playroom']
frame_num = np.arange(32)
camera_types = ['lietorch']
solving_methods = ['coarse']
init_render_resolutions = [1]

In [34]:
combinations = []
for start_frame in frame_num:
    for camera_type in camera_types:
            for room_name in room_names:
                for solving_method in solving_methods:
                    for init_render_resolution in init_render_resolutions:
                        combinations.append([room_name, start_frame, start_frame+1, camera_type, solving_method, init_render_resolution])

In [35]:
len(combinations)

64

In [36]:
# combinations += combinations[-16:]
combinations = np.array(combinations).reshape(8,1, 8, 6)

In [ ]:
output_bash_dir = '/mnt/sdb1/home/kbotashev/iros_paper/ibr_dataset/results/input_bashs'
min_gpu = 0
terminal_lines = []
for gpu_id, combo6 in enumerate(combinations):
    for gpu_sub_id, combo3 in enumerate(combo6):
        lines = []
        for combo in combo3:
            lines.append('\nCUDA_VISIBLE_DEVICES='+str(gpu_id+min_gpu) + 
                  ', /mnt/sdb1/home/kbotashev/anaconda3/envs/gs102/bin/python' + 
                  ' /mnt/sdb1/home/kbotashev/mip-nerf_projects/gaussian_splatting_original/gaussian-splatting/solve_pose_estimation.py ' + \
                    '--room_name='+combo[0] + ' --start_frame='+combo[1] + ' --end_frame='+combo[2] + \
                    ' --camera_type='+combo[3] + ' --solving_method='+combo[4] + ' --init_render_resolution='+combo[5])
        f = open(os.path.join(output_bash_dir, str(gpu_id+min_gpu) + '_' + str(gpu_sub_id) + ".sh"), "w")
        f.writelines(lines)
        f.close()
        terminal_lines.append('\nnohup bash ' + os.path.join(output_bash_dir, str(gpu_id+min_gpu) + 
                                                           '_' + str(gpu_sub_id) + ".sh") + ' > ' + 
                                                           os.path.join(output_bash_dir, str(gpu_id+min_gpu) + '_' + str(gpu_sub_id) + '_log.out &'))

f = open(os.path.join(output_bash_dir, "run_bashes.sh"), "w")
f.writelines(terminal_lines)
f.close()

In [13]:
output_bash_dir = '/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/results_task_rebase_vladnet/input_bashs'
min_gpu = 0
terminal_lines = []
for gpu_id, combo6 in enumerate(combinations):
    for gpu_sub_id, combo3 in enumerate(combo6):
        lines = []
        for combo in combo3:
            lines.append('\nCUDA_VISIBLE_DEVICES='+str(gpu_id+min_gpu) + 
                  ', /mnt/sdb1/home/kbotashev/anaconda3/envs/gs102/bin/python' + 
                  ' /mnt/sdb1/home/kbotashev/mip-nerf_projects/gaussian_splatting_original/gaussian-splatting/solve_pose_estimation.py ' + \
                    '--room_name='+combo[0] + ' --start_frame='+combo[1] + ' --end_frame='+combo[2] + \
                    ' --camera_type='+combo[3] + ' --solving_method='+combo[4] + ' --init_render_resolution='+combo[5])
        f = open(os.path.join(output_bash_dir, str(gpu_id+min_gpu) + '_' + str(gpu_sub_id) + ".sh"), "w")
        f.writelines(lines)
        f.close()
        terminal_lines.append('\nnohup bash ' + os.path.join(output_bash_dir, str(gpu_id+min_gpu) + 
                                                           '_' + str(gpu_sub_id) + ".sh") + ' > ' + 
                                                           os.path.join(output_bash_dir, str(gpu_id+min_gpu) + '_' + str(gpu_sub_id) + '_log.out &'))

f = open(os.path.join(output_bash_dir, "run_bashes.sh"), "w")
f.writelines(terminal_lines)
f.close()

: 